In [ ]:
!pip install -Uqq ultralytics

In [ ]:
from ultralytics import YOLO
from pathlib import Path
import os
import random
from PIL import Image, ImageDraw, ImageFont
import yaml
import matplotlib.pyplot as plt
import wandb
import cv2

In [ ]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("wandb_key")

wandb.login(key = secret_value_0)

In [ ]:
def load_classes_from_yaml(yaml_path):
    with open(yaml_path, 'r') as f:
        data = yaml.safe_load(f)
    return data['names']

In [ ]:
def get_yolo_annotations(image_path, txt_path, yaml_path, box_color='red', thickness=2):
    class_names = load_classes_from_yaml(yaml_path)
    image = Image.open(image_path).convert("RGB")
    draw = ImageDraw.Draw(image)
    w, h = image.size

    with open(txt_path, 'r') as f:
        lines = f.readlines()

    for line in lines:
        cls, x_center, y_center, width, height = map(float, line.strip().split())

        x1 = (x_center - width / 2) * w
        y1 = (y_center - height / 2) * h
        x2 = (x_center + width / 2) * w
        y2 = (y_center + height / 2) * h
        draw.rectangle([x1, y1, x2, y2], outline=box_color, width=thickness)
        draw.text((x1, y1 - 10), class_names[int(cls) -1], fill=box_color)

    return image

In [ ]:
def visualize_yolo_data(data_path: Path):
    images_path = data_path / 'images'
    labels_path = data_path / 'labels'

    random_images_paths = random.sample(list(images_path.glob('*.jpg')), 10)
    labels_paths = [labels_path / (img.stem + '.txt') for img in random_images_paths]

    fig, axs = plt.subplots(nrows=2, ncols=5, figsize=(12, 8))
    axs = axs.flatten()

    for ax, image_path, txt_path in zip(axs, random_images_paths, labels_paths):
        annotated_image = get_yolo_annotations(str(image_path), str(txt_path), '/kaggle/input/garbage-data/YOLO-Waste-Detection-1/YOLO-Waste-Detection-1/data.yaml')
        ax.imshow(annotated_image)
        ax.axis('off')

    plt.tight_layout()

visualize_yolo_data(Path('/kaggle/input/garbage-data/YOLO-Waste-Detection-1/YOLO-Waste-Detection-1/train'))

In [ ]:
model = YOLO('yolo11l.pt', task = 'detect')

In [ ]:
model.train(data = '../input/garbage-data/data.yaml', time = 10, cos_lr = True,  batch = -1, imgsz = 640, project = 'dsn ai project', name = 'yolo11l')